In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as data_reader
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [8]:
company = "FB"

start = dt.datetime(2012,1,1)
end = dt.datetime(2020,1,1)

data = data_reader.DataReader(company, 'yahoo', start, end)

data.head()
#data.info()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-05-18,45.000000,38.000000,42.049999,38.230000,573576400,38.230000
2012-05-21,36.660000,33.000000,36.529999,34.029999,168192700,34.029999
2012-05-22,33.590000,30.940001,32.610001,31.000000,101786600,31.000000
2012-05-23,32.500000,31.360001,31.370001,32.000000,73600000,32.000000
2012-05-24,33.209999,31.770000,32.950001,33.029999,50237200,33.029999


In [13]:
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

print (scaled_data)

[[0.10261801]
 [0.08159383]
 [0.06642639]
 ...
 [0.95294592]
 [0.93447466]
 [0.93867948]]


In [24]:
prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
  x_train.append(scaled_data[x-prediction_days:x, 0])
  y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

print(x_train.shape)

(1857, 60, 1)


In [25]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape = (x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
#prediction of the next closing value
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
59/59 [==============================] - 10s 86ms/step - loss: 0.0794
Epoch 2/25
59/59 [==============================] - 6s 96ms/step - loss: 0.0049
Epoch 3/25
59/59 [==============================] - 6s 94ms/step - loss: 0.0037
Epoch 4/25
59/59 [==============================] - 6s 96ms/step - loss: 0.0037
Epoch 5/25
59/59 [==============================] - 6s 96ms/step - loss: 0.0065
Epoch 6/25
59/59 [==============================] - 6s 96ms/step - loss: 0.0040
Epoch 7/25
59/59 [==============================] - 6s 96ms/step - loss: 0.0102
Epoch 8/25
59/59 [==============================] - 6s 95ms/step - loss: 0.0031
Epoch 9/25
59/59 [==============================] - 6s 97ms/step - loss: 0.0027
Epoch 10/25
59/59 [==============================] - 6s 96ms/step - loss: 0.0037
Epoch 11/25
59/59 [==============================] - 6s 100ms/step - loss: 0.0029
Epoch 12/25
59/59 [==============================] - 6s 98ms/step - loss: 0.0029
Epoch 13/25
59/59 [================